In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku_biweekly (
  calday DATE,
  roll_out STRING,
  region STRING,
  mch3_id STRING,
  mch3_name STRING,
  product_id STRING,
  product_name STRING,
  manufacturer_id STRING,
  manufacturer_name STRING,
  sub_manufacturer STRING,
  uom STRING,
  assortment STRING,
  vendor_id STRING,
  vendor_name STRING,
  vendor_subrange STRING,
  is_out STRING
)

TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
          DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku_biweekly
	WHERE CALDAY = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
          """)

In [0]:

spark.sql(f"""INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku_biweekly
	SELECT 
		calday,
		roll_out,
		region,
		mch3_id,
		mch3_name,
		product_id,
		product_name,
		manufacturer_id,
		manufacturer_name,
		sub_manufacturer,
		uom,
		assortment,
		vendor_id,
		vendor_name,
		vendor_subrange,
		"" is_out
	FROM udp_wcm_dev.udp_wcm_gold_vcm_dwh.d_vendor_sku_his 
	WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)""")